# Byte Pair Encoding

Tokens are the atoms of large language models.
Tokenization is the process of translating the text into sequences of tokens.


Tokenization is at the heart of much weridness of LLMs.

- Why can't LLMs spell words? Tokenization

- Why can't LLMs do super simple string processing tasks like reversing a string? Tokenization

- Why are LLMs worse at non-English languages? Tokenization 

- Why is LLM bad at arithmetic? Tokenization

- Why did GPT-2 have more than necessary trouble coding in python? Tokenization

- Why did my LLM abruptly halt when it sees the string <|endoftext|>? Tokenization

- What is this weird warning I get about a "trailing whitespace"? Tokenization

- Why the LLM breaks if I ask it about "SolidGoldMagikarp"? Tokenization

- Why should I prefer to use YAML over JSON with LLMs? Tokenization

- Why is LLM not actually end-to-end language modeling? Tokenization

- What is the real root of suffering? Tokenization.

Strings are immutable sequences of Unicode code points.(what integers represent all the characters defined in the Unicode)

In [2]:
ord("h") # the unicode code point in Python for a single character not words
ord("ز")

1586

Why can't we just use Unicode code points natively and not use the tokenization at all?

1) The vocabulary will be quite long.(149813 characters)

2) This standard is very much alive and keep changing, so it is not a stable representation.


Encoding is a way to get the unicode text and translate it to byte streams or binary data.

This byte stream is between 1 and 4 bytes for UTF-8.

UTF-8 is preferred over UTF-16 and UTF-32.




In [3]:
"زینب".encode("utf-8") # b is bytes 

b'\xd8\xb2\xdb\x8c\xd9\x86\xd8\xa8'

In [4]:
list("زینب".encode("utf-8"))

[216, 178, 219, 140, 217, 134, 216, 168]

In [5]:
list("زینب".encode("utf-16")) # we have a bunch of 6s which indicates a waste of encoding and that's why this encoding is not preferred.

[255, 254, 50, 6, 204, 6, 70, 6, 40, 6]

# if we use UTF-8 naively, these are byte streams so that would imply the vocabulary of length 256. This size is quite small. This won't allow us to attend to sufficiently long text before the curretn token since our sequences will be long.
So, we don't want to use the raw bytes of the UTF-8 encoding!
We are in favor of feeding the byte streams to LLMs to get rid of the tokenization process.
We need to compress those byte pairs using the byte pair encoding algorithm.

### How does the Byte Pair Encoding algorithm work?

Replace the most occured pair with a new byte that is not in the data.

aaabdaaabac => Z=aa => ZabdZabac => Y=ab => ZYdZYac => X=ZY => XdXac

Our vocabulary is: {a, b, c, d, X, Y, Z}

This data can not be compressed further by byte pair encoding because there are no pairs of bytes that occur more than once.



In [19]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens)) # convert to a list of integers between 0 and 255 for convenience

print(text)
print(len(text))
print(tokens)
print(len(tokens))

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
533
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135,

In [20]:
# now find the pair of bytes that occur most frequently
def get_stats(tokens: list):
    counts = {}
    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


stats = get_stats(tokens)
print(sorted(((v,k) for k, v in stats.items()), reverse=True))

[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (

In [21]:
chr(101), chr(32)

('e', ' ')

After finding the most occured pair, we iterate through the sequence and mint that pair with a new token with the id of 256.

In [22]:
def merge_tokens(pair: tuple, tokens: list):
    vocab = range(256)
    outputs = tokens
    i = 0 
    while i < len(tokens):
        if tokens[i:i+2] == list(pair):
            outputs.remove(tokens[i+1])
            new_id = len(vocab)
            outputs[i] = new_id
            vocab.append(new_id)
            i += 1
    return outputs

In [23]:
merge_tokens((101, 32), tokens)

KeyboardInterrupt: 